![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F08+-+R&file=R+-+Working+With+BigQuery.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/08%20-%20R/R%20-%20Working%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F08%2520-%2520R%2FR%2520-%2520Working%2520With%2520BigQuery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/R%20-%20Working%20With%20BigQuery.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/08%20-%20R/R%20-%20Working%20With%20BigQuery.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# R - Working With BigQuery

Working with BigQuery and getting data from BigQuery into R.  This workflow covers multiple ways - all from R.

---
Part of the series of [**R**](https://github.com/statmike/vertex-ai-mlops/blob/main/08%20-%20R/readme.md) workflows:

A series of workflows focused on using **R** in Vertex AI as well as other Google Cloud services to run R code, train models with R, and serve predictionns with R.

---

**The Data**

The source data is first exported to Google Cloud Storage in CSV format below.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.    
- The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).
- Read mode about BigQuery public datasets [here](https://cloud.google.com/bigquery/public-data)

**Description of the Data**

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
>**Quick Note on PCA**<p>PCA is an unsupervised learning technique: there is not a target variable.  PCA is commonly used as a variable/feature reduction technique.  If you have 100 features then you could reduce it to a number p (say 10) projected features.  The choice of this number is a balance of how well it can explain the variance of the full feature space and reducing the number of features.  Each projected feature is orthogonal to each other feature, meaning there is no correlation between these new projected features.</p>

---

**Prerequisites:**

- This notebook running in Vertex AI Workbench Instance as described in the series [readme](./readme.md)

---
## Setup

inputs:

In [1]:
project_id <- system('gcloud config get-value project', intern = TRUE)
project_id

[1] "statmike-mlops-349915"

In [2]:
region <- 'us-central1'
experiment <- 'bigquery-data'
series <- 'r'

# BigQuery Parameters
bq_project <- project_id
bq_dataset <- series
bq_table <- experiment
bq_region <- substr(region, 1, 2)
bq_source <- 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

# GCS Parameters: Give bucket name
gcs_bucket <- project_id

# key columns in the data:
var_target <- 'Class'
var_omit <- list('transaction_id', 'splits')

packages:

In [4]:
library(bigrquery)
library(DBI)
library(dplyr)

---

## Reading From BigQuery

In using the `bigrquery` package there are three ways to retrieve data from BigQuery.  These are covered below and are good for smaller data sizes (<100MB).  An alternative is also covered that works for any data size.

- Reference for [`bigrquery`](https://bigrquery.r-dbi.org/)

### Retrive with `bigrquery`

A wrapper for the BigQuery REST API.  Provides a set of [functions](https://sites.google.com/corp/google.com/gemini-ultra-customer-access/home) that all start with 'bq_'.

Define the query that reads the table:

In [5]:
query <- sprintf('
    SELECT *
    FROM `%s`
    LIMIT 5
', bq_source)
cat(query)


    SELECT *
    FROM `bigquery-public-data.ml_datasets.ulb_fraud_detection`
    LIMIT 5


Connect to the table defined by the query:

In [6]:
table <- bq_project_query(bq_project, query)

Load records from the table:

In [7]:
ds1 <- bq_table_download(table, n_max = 2)

Review the size and preview the records:

In [8]:
dim(ds1)

[1]  2 31

In [9]:
ds1

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
8748,-1.070416,0.3045173,2.7770644,2.154061,0.2544503,-0.4485295,-0.39869079,0.14467154,1.070900,⋯,-0.12203215,-0.18235147,0.01957590,0.6260233,-0.01851757,-0.26329053,-0.19860014,0.09843517,0,0
27074,1.165628,0.4236713,0.8876348,2.740163,-0.3385779,-0.1428458,-0.05562784,-0.01532457,-0.213621,⋯,-0.08118352,-0.02569424,-0.07660875,0.4146867,0.63103248,0.07732208,0.01018155,0.01991206,0,0


### Retrieve with `DBI`

An easier interface to BigQuery that reads the results of a query directly.

Create a connection:

In [10]:
connection <- dbConnect(
    bigrquery::bigquery(),
    project = bq_project,
    billing = bq_project
)

Load records from table using the connection:

In [11]:
ds2 <- dbGetQuery(connection, query, n = 2)

Review the size and preview the records:

In [12]:
dim(ds2)

[1]  2 31

In [13]:
ds2

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
8748,-1.070416,0.3045173,2.7770644,2.154061,0.2544503,-0.4485295,-0.39869079,0.14467154,1.070900,⋯,-0.12203215,-0.18235147,0.01957590,0.6260233,-0.01851757,-0.26329053,-0.19860014,0.09843517,0,0
27074,1.165628,0.4236713,0.8876348,2.740163,-0.3385779,-0.1428458,-0.05562784,-0.01532457,-0.213621,⋯,-0.08118352,-0.02569424,-0.07660875,0.4146867,0.63103248,0.07732208,0.01018155,0.01991206,0,0


### Retrieve with `dplyr`

A non-SQL interface to BigQuery that treats BigQuery tables like they are already in-memory and local.  Behind the scenes it writes SQL for you.

Create a connection to BigQuery:

In [18]:
connection <- dbConnect(
    bigrquery::bigquery(),
    project = strsplit(bq_source, '[.]')[[1]][1],
    dataset = strsplit(bq_source, '[.]')[[1]][2],
    billing = bq_project
)

In [19]:
extract <- tbl(connection, strsplit(bq_source, '[.]')[[1]][3])

In [20]:
glimpse(extract)

Rows: ??
Columns: 31
Database: BigQueryConnection
$ Time   <dbl> 8748, 27074, 28292, 28488, 31392, 31887, 34482, 34974, 35765, 4…
$ V1     <dbl> -1.0704164, 1.1656278, 1.0508789, 1.0703157, -3.6809531, -0.861…
$ V2     <dbl> 0.30451734, 0.42367133, 0.05340835, 0.07949856, -4.18358129, 0.…
$ V3     <dbl> 2.7770644, 0.8876348, 1.3645902, 1.4718556, 2.6427427, 2.410561…
$ V4     <dbl> 2.1540615, 2.7401630, 2.6661578, 2.8637862, 4.2638021, 3.317876…
$ V5     <dbl> 0.254450301, -0.338577868, -0.378635805, -0.637886843, 4.643285…
$ V6     <dbl> -0.4485295, -0.1428458, 1.3820315, 0.8581592, -0.2250528, 0.772…
$ V7     <dbl> -0.39869079, -0.05562784, -0.76620210, -0.68747791, -3.73363726…
$ V8     <dbl> 0.14467154, -0.01532457, 0.48612560, 0.34414609, 1.27303676, 0.…
$ V9     <dbl> 1.07090041, -0.21362101, 0.15261147, 0.45956062, 0.01566150, -0…
$ V10    <dbl> -0.31050073, 0.51726104, 0.49667421, 0.34086927, 0.63552321, 0.…
$ V11    <dbl> 0.09397115, -0.99559462, 0.34198230, -1.14145224, -0.44

In [21]:
ds3 <- extract %>%
    select(everything()) %>%
    head(2) %>%
    collect()

Review the size and preview the records:

In [22]:
dim(ds3)

[1]  2 31

In [23]:
ds3

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
8748,-1.070416,0.3045173,2.7770644,2.154061,0.2544503,-0.4485295,-0.39869079,0.14467154,1.070900,⋯,-0.12203215,-0.18235147,0.01957590,0.6260233,-0.01851757,-0.26329053,-0.19860014,0.09843517,0,0
27074,1.165628,0.4236713,0.8876348,2.740163,-0.3385779,-0.1428458,-0.05562784,-0.01532457,-0.213621,⋯,-0.08118352,-0.02569424,-0.07660875,0.4146867,0.63103248,0.07732208,0.01018155,0.01991206,0,0


## Retrieve Large Tables

Downloading large table to **R** with `bigrquery` is better as a two step process: download to GCS, then read into R.  

>This is also mentioned in the [`bigrquery` documentation](https://bigrquery.r-dbi.org/reference/bq_table_download.html), however, this mention a middle step of copying the file from GCS to local which is skipped here thanks to the Fuse mount.  

This section show how to orchestrate this all from **R**.

### Preparation: Create Fuse Mount For Workbench Instance

When using a Vertex AI Workbench the instance already has GCS Fuse installed.  Use the following steps at a terminal (File > New > Terminal) to mount a local folder for use:

```
cd ~/
mkdir -p gcs
gcsfuse --implicit-dirs --rename-dir-limit=100 --max-conns-per-host=100 "/home/jupyter/gcs"
```

> When running a Vertex AI Training job this mount is already done for you at the `/gcs/` location.

### Export Data From BigQuery To GCS

Use a SQL Query to initiate and export from BigQuery to GCS at a specified location.  This example show exporting to `.csv` files with a wildcard.  The example data is small enough that is outputs to just two files but this code would work with much larger datasets that export to multiple `.csv` files as well.

**Resources:**

- [BigQuery data exports](https://cloud.google.com/bigquery/docs/exporting-data#sql)

Define the query that reads the table:

In [47]:
query <- sprintf("
EXPORT DATA
    OPTIONS (
        uri = 'gs://%s/%s/%s/data/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT *
    FROM `%s`
)
", gcs_bucket, series, experiment, bq_source)
cat(query)


EXPORT DATA
    OPTIONS (
        uri = 'gs://statmike-mlops-349915/r/bigquery-data/data/*.csv',
        format = 'CSV',
        overwrite = true,
        header = true,
        field_delimiter = ','
    )
AS (
    SELECT *
    FROM `bigquery-public-data.ml_datasets.ulb_fraud_detection`
)


Run the query using `bigrquery`:

In [48]:
export <- bq_perform_query(query, billing = bq_project)

Wait on the export job to complete:

In [49]:
bq_job_wait(export)

### List The Exported Files:

In [50]:
files <- list.files(
    path = sprintf('~/gcs/%s/%s/%s/data', gcs_bucket, series, experiment),
    pattern = '*.csv',
    full.names = TRUE
)
files

[1] "/home/jupyter/gcs/statmike-mlops-349915/r/bigquery-data/data/000000000000.csv"
[2] "/home/jupyter/gcs/statmike-mlops-349915/r/bigquery-data/data/000000000001.csv"

### Read the Data Into An R Dataframe

The code here will read all `.csv` files at the bucket folder path into a single R dataframe.  Here all the files are read into a single dataframe using the `map_df()` function which also executes the reads in parallel (asynchronously).

Load the [`purrr`](https://purrr.tidyverse.org/) library:

In [56]:
library(purrr)

In [55]:
df <- 
    files %>%
    map_df(~fread(.))

Review the size and preview the records:

In [57]:
dim(df)

[1] 284807     31

In [58]:
head(df, 2)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
8748,-1.070416,0.3045173,2.7770644,2.154061,0.2544503,-0.4485295,-0.39869079,0.14467154,1.070900,⋯,-0.12203215,-0.18235147,0.01957590,0.6260233,-0.01851757,-0.26329053,-0.19860014,0.09843517,0,0
27074,1.165628,0.4236713,0.8876348,2.740163,-0.3385779,-0.1428458,-0.05562784,-0.01532457,-0.213621,⋯,-0.08118352,-0.02569424,-0.07660875,0.4146867,0.63103248,0.07732208,0.01018155,0.01991206,0,0


### Remove Exported Data From GCS

Optionally, delete the data exported from BigQuery to GCS.  Depending on your workflow you may want to adopt the practice of cleaning up the exported data with each run of this code when it is finished.

In [59]:
#sapply(files, unlink)

---
## Prepare Data For Models

While **R** is an excellent tool for preparing data for machine learning it can make serving the resulting models challenging if the same processing needs to occur on future data.  This section shows how to use **R** to orchestrate some preliminary data preparation steps in BigQuery and then load the results into **R** using the methods presented above.

### BigQuery Dataset

In BigQuery, tables are arranged in groups called datasets that are resources within Google Cloud projects.  This three level organization make it easy to refer to data table (or views).  

Create a dataset within the current project to hold a prepared version of the data.  Start by checking to see if it already exists.

- Reference: [BigQuery datasets with R/bq-datasets.R](https://bigrquery.r-dbi.org/reference/api-dataset.html)

Create BigQuery dataset object:

In [61]:
bq_ds <- bq_dataset(bq_project, bq_dataset)

Check for existance of the dataset, create if needed:

In [68]:
if (bq_dataset_exists(bq_ds)) {
    print('Dataset already exists')
} else {
    print('Creating dataset')
    bq_dataset_create(bq_ds, location = bq_region)
}

[1] "Creating dataset"


<bq_dataset> statmike-mlops-349915.r

### Create Table

Create a copy of the source table in the new dataset and add row leve id's (`transacation_id`) and assign splits for Train/Test (`splits`).

Define the query that creates the table:

In [73]:
query <- sprintf('
CREATE TABLE IF NOT EXISTS `%s.%s.%s` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `%s`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
', bq_project, bq_dataset, bq_table, bq_source)
cat(query)


CREATE TABLE IF NOT EXISTS `statmike-mlops-349915.r.bigquery-data` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `bigquery-public-data.ml_datasets.ulb_fraud_detection`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id


Run the query using `bigrquery`:

In [74]:
create <- bq_perform_query(query, billing = bq_project)

Wait on the create job to complete:

In [75]:
bq_job_wait(create)

### Retrieve Table

Using the `bigrquery` method from above, retrieve the full table to a dataframe.

Define the query that reads the table.  Take advantage of BigQuery columnar data by excluding columns that are not needed while also using a `WHERE` statment to filter to rows allocated for model training.  In the inputs above a list of these was created named `var_omit`.

In [87]:
query <- sprintf('
    SELECT * EXCEPT(%s)
    FROM `%s.%s.%s`
    WHERE splits = "TRAIN"
', paste(unlist(var_omit), collapse = ','),
bq_project, bq_dataset, bq_table)
cat(query)


    SELECT * EXCEPT(transaction_id,splits)
    FROM `statmike-mlops-349915.r.bigquery-data`
    WHERE splits = "TRAIN"


Connect to the table defined by the query:

In [88]:
table <- bq_project_query(bq_project, query)

Load records from the table:

In [89]:
ds <- bq_table_download(table, n_max = Inf)

Review the size and preview the records:

In [90]:
dim(ds)

[1] 227879     31

In [91]:
head(ds, 2)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
5043,-0.6103529,0.8762678,3.134572,2.26016851,0.001184993,0.2684391,0.1270940,-0.008680134,0.9528023,⋯,-0.2022804,-0.1228932,-0.1831320,0.2959793,-0.15998880,-0.1301962,-0.076139183,-0.109075941,0,0
43968,1.1032424,-0.4789847,1.136295,-0.05461861,-0.823168828,0.7920736,-0.9883738,0.492957197,0.8836801,⋯,-0.0200895,0.1700578,0.1209046,-0.2092878,-0.07528029,1.0339329,-0.005642291,-0.002844234,0,0
